In [ ]:
import keras
%matplotlib inline
import matplotlib.pyplot as plt



# Introducción a las convnets: Clasificando números

Esta libreta contiene un ejemplo sencillo que permite clasificar número escritos a mano sobre fondo negro.


Vamos a echarle un vistazo a un ejemplo sencillo de una red convolucional (convnet). La utilizaremos para clasificar el dataset MNIST, que es un dataset abierto que contiene números escritos a mano. 

![Números escritos a mano del dataset MNIST](http://corochann.com/wp-content/uploads/2017/02/mnist_plot.png)

Las 6 líneas de código que puedes ver aqui debajo muestran que aspecto tiene una convnet basica. Es una pila de capas `Conv2D` y `MaxPooling2D`. Veremos en un rato más concretamente lo que hacen. 
Lo importante es notar que una convnet toma como input tensores de tamaño `(altura_imagen, anchura_imagen, canales_imagen)`. 
En nuestro caso, vamos a configurar nuestra convnet para procesar inputs de tamaño `(28, 28, 1)`,  que es el formato de las imágenes MNIST. 
Por eso le pasamos el argumento input_shape=(28, 28, 1)` a nuestra primera capa.

In [ ]:
from keras import layers
from keras import models
from keras.layers import Conv2D


model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Vamos a ver que pinta tiene nuestra arquitectura hasta ahora:

In [ ]:
model.summary()

Puedes ver arriba que la salida de cada capa `Conv2D` y `MaxPooling2D` es un tensor 3D de dimensiones `(altura, anchura, canales)`. La anchura y la altura tienden a diminuir según vamos yendo mas profundo en la red. El número de canales está controlado por el primer argumento que se le pasa a 
las capas `Conv2D`  (e.j. 32 o 64).

El siguiente paso sería darle nuestro ultimo tensor (de dimensiones `(3, 3, 64)`) como entrada a una red densamente conectada. 
Estos clasificadores procesan vectores, que son 1D,  mientras que nuestra salida es un tensor 3D. 
Así que primero tendremos que aplanar nuestra salida 3D y convertirla en 1D y después añadir unas cuantas capas densas:

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Queremos clasificar 10 categorías, lo que significa que nuestra capa final debe tener 10 nodos y una función de activación softmax. Vamos a ver que pinta tiene nuestra red:

In [ ]:
model.summary()

Como puedes ver, nuestra salida de dimension `(3, 3, 64)` han sido aplanadas hasta convertirse en vectores de dimensión `(576,)`, antes de entrar en las dos capas densas.

Vamos ahora a entrenar nuestra red con las imágenes del dataset MNIST. 


In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("Tamaño del dataset de training antes de preprocesarlo: ", train_images.shape, "\n")
print ("Etiqueta de la primera imagen del dataset: ", train_labels[0], "\n")
print ("Primer elemento del dataset en formato matriz: \n \n", train_images[0], "\n")
print("Primer elemento del dataset en formato imagen: ", plt.imshow(train_images[0],cmap='Greys'))



In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print("Shape del dataset de training después del preprocesado: ", train_images.shape)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1, batch_size=64)

Vamos a evaluar el modelo con las imágenes de test que la red no ha visto nunca antes:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
print("Esta es la precisión sobre las imágenes del test: ", test_acc)

¡No está mal! 


In [ ]:
from keras.preprocessing import image
import numpy as np
from matplotlib import pyplot as plt

img_width=28
img_height=28

img = image.load_img('uno.png', target_size=(img_width, img_height),grayscale=True)
plt.imshow(img,cmap='Greys')
x= image.img_to_array(img)
x = np.expand_dims(x, axis=0)
print("La imagen representa el número: ", model.predict_classes(x))